In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


<font color='tomato'><font color="#CC3D3D"><p>
# A DNN Model for Multiclass Classification

##### Import modules

In [ ]:
!pip install -q -U keras-tuner

     |████████████████████████████████| 133 kB 8.9 MB/s 


In [ ]:
import pandas as pd
import numpy as np
import os
import random
import pickle
from tqdm import tqdm
from IPython.display import Image, clear_output
import seaborn as sns
import matplotlib.pylab as plt
from matplotlib import font_manager, rc
%matplotlib inline

from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow import keras
import kerastuner as kt
print(tf.__version__)

2.8.2


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  app.launch_new_instance()


##### Set random seeds to make your results reproducible

In [ ]:
# 매번 모델링을 할 때마다 동일한 결과를 얻으려면 아래 코드를 실행해야 함.

def reset_seeds(s1,s2,s3, reset_graph_with_backend=None):
    if reset_graph_with_backend is not None:
        K = reset_graph_with_backend
        K.clear_session()
        tf.compat.v1.reset_default_graph()
        print("KERAS AND TENSORFLOW GRAPHS RESET")  # optional

    np.random.seed(s1)
    random.seed(s2)
    tf.compat.v1.set_random_seed(s3)
    os.environ['CUDA_VISIBLE_DEVICES'] = ''  # for GPU
#    print("RANDOM SEEDS RESET")  # optional

### Step 1: Load and process the data

##### Read data

In [ ]:
# 학습데이터에 custid가 없으면 추가
IDtest = pd.DataFrame({'custid': pd.read_csv('/content/drive/MyDrive/머러컴피티션/X_test.csv', encoding = "cp949").custid.unique()})

In [ ]:
# 데이터불러오기

In [ ]:
df_train = pd.read_csv('/content/drive/MyDrive/머러컴피티션/진짜마지막/진짜마지막_train.csv')

In [ ]:
df_train

,1,0,169,231,4,3,2,173,352,25,...,839,34,841,1320,1155,911,873,1345,535,1141
0,-0.401774,0.258656,0.616024,0.242885,0.059594,0.008479,1.098612,-0.002424,-0.517807,19.000000,...,-0.107617,-0.152542,0,-0.217619,0.0,-0.196841,-0.083853,-0.173885,-0.218596,-0.078677
1,-0.347583,0.258891,0.380067,0.241397,0.028305,0.055183,1.098612,-0.010025,0.272619,14.142857,...,-0.107617,9.593732,0,-0.217619,0.0,-0.196841,-0.083853,-0.173885,-0.218596,-0.078677
2,-0.017115,0.260489,-0.710264,0.234344,-0.003134,-0.086416,1.098612,0.015999,-0.227984,16.000000,...,-0.107617,-0.152542,0,-0.217619,0.0,-0.196841,-0.083853,-0.173885,0.164404,-0.078677
3,0.103395,0.260940,-1.058357,0.232445,-0.033140,-0.061537,0.693147,0.027703,-0.517807,14.000000,...,-0.107617,-0.152542,0,-0.217619,0.0,-0.196841,-0.083853,-0.173885,-0.218596,-0.078677
4,-0.456512,0.258369,0.827803,0.244247,0.032277,0.086695,1.386294,0.016045,0.840738,15.444444,...,-0.107617,-0.152542,0,-0.217619,0.0,-0.196841,-0.083853,-0.173885,-0.218596,-0.078677
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21582,-0.260748,0.259313,0.315357,0.241041,0.003974,-0.072648,1.098612,0.010104,-0.517807,15.060000,...,-0.107617,-0.152542,0,-0.217619,5.0,11.223432,-0.083853,-0.173885,-0.218596,-0.078677
21583,-1.113412,0.254983,0.261953,0.240611,-0.072966,-0.076272,0.693147,-0.029911,-0.517807,14.625000,...,-0.107617,-0.152542,6,-0.217619,0.0,-0.196841,-0.083853,-0.173885,-0.218596,-0.078677
21584,0.570301,0.263636,-0.537937,0.236095,-0.010159,-0.035039,1.098612,0.003240,-0.517807,13.500000,...,-0.107617,-0.152542,0,-0.217619,1.0,-0.196841,-0.083853,-0.173885,-0.218596,-0.078677
21585,-0.745471,0.256832,0.384082,0.241566,-0.013758,-0.121351,0.693147,0.024548,-0.517807,14.500000,...,-0.107617,-0.152542,0,-0.217619,0.0,-0.196841,-0.083853,-0.173885,-0.218596,-0.078677


In [ ]:
df_test = pd.read_csv('/content/drive/MyDrive/머러컴피티션/진짜마지막/진짜마지막_test.csv')

In [ ]:
y_train = pd.read_csv('/content/drive/MyDrive/머러컴피티션/y_train.csv').group

##### One-hot-encode Target variable 

In [ ]:
# 8개의 범주형 타겟 값을 one-hot-encoding을 통해 8개의 컬럼으로 만들어야 함.
y_train = keras.utils.to_categorical(y_train.astype('category').cat.codes)

In [ ]:
df_train

,1,0,169,231,4,3,2,173,352,25,...,839,34,841,1320,1155,911,873,1345,535,1141
0,-0.401774,0.258656,0.616024,0.242885,0.059594,0.008479,1.098612,-0.002424,-0.517807,19.000000,...,-0.107617,-0.152542,0,-0.217619,0.0,-0.196841,-0.083853,-0.173885,-0.218596,-0.078677
1,-0.347583,0.258891,0.380067,0.241397,0.028305,0.055183,1.098612,-0.010025,0.272619,14.142857,...,-0.107617,9.593732,0,-0.217619,0.0,-0.196841,-0.083853,-0.173885,-0.218596,-0.078677
2,-0.017115,0.260489,-0.710264,0.234344,-0.003134,-0.086416,1.098612,0.015999,-0.227984,16.000000,...,-0.107617,-0.152542,0,-0.217619,0.0,-0.196841,-0.083853,-0.173885,0.164404,-0.078677
3,0.103395,0.260940,-1.058357,0.232445,-0.033140,-0.061537,0.693147,0.027703,-0.517807,14.000000,...,-0.107617,-0.152542,0,-0.217619,0.0,-0.196841,-0.083853,-0.173885,-0.218596,-0.078677
4,-0.456512,0.258369,0.827803,0.244247,0.032277,0.086695,1.386294,0.016045,0.840738,15.444444,...,-0.107617,-0.152542,0,-0.217619,0.0,-0.196841,-0.083853,-0.173885,-0.218596,-0.078677
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21582,-0.260748,0.259313,0.315357,0.241041,0.003974,-0.072648,1.098612,0.010104,-0.517807,15.060000,...,-0.107617,-0.152542,0,-0.217619,5.0,11.223432,-0.083853,-0.173885,-0.218596,-0.078677
21583,-1.113412,0.254983,0.261953,0.240611,-0.072966,-0.076272,0.693147,-0.029911,-0.517807,14.625000,...,-0.107617,-0.152542,6,-0.217619,0.0,-0.196841,-0.083853,-0.173885,-0.218596,-0.078677
21584,0.570301,0.263636,-0.537937,0.236095,-0.010159,-0.035039,1.098612,0.003240,-0.517807,13.500000,...,-0.107617,-0.152542,0,-0.217619,1.0,-0.196841,-0.083853,-0.173885,-0.218596,-0.078677
21585,-0.745471,0.256832,0.384082,0.241566,-0.013758,-0.121351,0.693147,0.024548,-0.517807,14.500000,...,-0.107617,-0.152542,0,-0.217619,0.0,-0.196841,-0.083853,-0.173885,-0.218596,-0.078677


In [ ]:
df_trains = df_train.iloc[:,1:]
df_tests = df_test.iloc[:,1:]

In [ ]:
X_train = df_train
X_test = df_test

##### Split data into train & validation set 

In [ ]:
i = int(round(X_train.shape[0] * 0.8,0))
X_valid, y_valid = X_train[i:], y_train[i:]
X_train, y_train = X_train[:i], y_train[:i]

In [ ]:
y_train

array([[0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.]], dtype=float32)

##### Feature scaling

In [ ]:
X_train.shape, X_valid.shape, X_test.shape

((17270, 1617), (4317, 1617), (14380, 1617))

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

In [ ]:
all = pd.concat([X_train,X_valid,X_test]).iloc[:,1:]

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
alls = scaler.fit_transform(all)
alll = pd.DataFrame(alls);alll

,0,1,2,3,4,5,6,7,8,9,...,1342,1343,1344,1345,1346,1347,1348,1349,1350,1351
0,-0.408293,-0.416531,0.651018,0.179373,2.327367,-0.774276,0.107623,3.935217,-0.680853,-0.763185,...,-1.927735,1.915716,-0.271203,-0.173885,-0.052019,1.811228,0.229993,-0.502118,-0.226102,0.054776
1,-0.360225,-0.360362,0.651018,0.674131,1.158107,-0.603846,-0.204984,2.358436,-0.761619,-0.694785,...,-0.406580,-0.156788,-1.240056,-0.173885,-0.052019,0.327239,-0.973965,-0.548124,-0.226102,0.047286
2,-0.032784,-0.017829,0.651018,-0.825886,-0.016756,-0.433416,-0.068172,0.117703,0.132541,0.711179,...,-0.625861,-1.038187,-1.117591,-0.173885,-0.052019,-1.652902,1.451063,0.064013,-0.226102,-0.619278
3,0.059696,0.107079,-0.935282,-0.562332,-1.138087,-0.547036,-0.201778,-0.401271,2.655044,0.017943,...,-0.288925,0.347900,0.628829,-0.173885,-0.052019,0.452675,-3.794050,-0.767526,-0.226102,-0.811634
4,-0.467137,-0.473268,1.776515,1.007943,1.306532,-0.433416,1.122939,2.038003,0.035973,-0.001205,...,-0.453735,-0.411870,-0.031423,-0.173885,-0.052019,-0.400859,0.820233,0.103539,-0.226102,-0.603137
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35962,2.434637,2.470233,-0.935282,2.444313,0.085558,-0.774276,-0.421931,1.044225,1.073118,-0.932893,...,-1.149085,0.184284,0.821311,-0.173885,-0.052019,-0.328714,0.572698,-0.772905,-0.226102,1.655081
35963,-0.357541,-0.357676,-0.935282,0.398871,3.101870,-0.717466,0.297125,1.044225,-2.228148,-0.818197,...,-2.168055,2.764155,0.481166,-0.173885,-0.052019,2.188832,-0.447706,-0.851536,-0.226102,0.737441
35964,-0.165235,-0.198549,-0.935282,-0.075443,1.098311,-0.774276,-0.238699,1.044225,-0.253429,-1.827729,...,-1.182235,2.204344,-0.363499,-0.173885,-0.052019,0.663401,0.070965,-0.809742,-0.226102,-0.254768
35965,3.405554,3.431798,-0.935282,2.000310,-0.181798,-0.717466,2.146455,-1.124019,2.033663,1.429454,...,-0.081216,-1.098615,0.837121,-0.173885,-0.052019,-0.854609,0.400064,-0.744621,-0.226102,-0.337506


In [ ]:
X_train = alll[:17270];X_train

,0,1,2,3,4,5,6,7,8,9,...,1342,1343,1344,1345,1346,1347,1348,1349,1350,1351
0,-0.408293,-0.416531,0.651018,0.179373,2.327367,-0.774276,0.107623,3.935217,-0.680853,-0.763185,...,-1.927735,1.915716,-0.271203,-0.173885,-0.052019,1.811228,0.229993,-0.502118,-0.226102,0.054776
1,-0.360225,-0.360362,0.651018,0.674131,1.158107,-0.603846,-0.204984,2.358436,-0.761619,-0.694785,...,-0.406580,-0.156788,-1.240056,-0.173885,-0.052019,0.327239,-0.973965,-0.548124,-0.226102,0.047286
2,-0.032784,-0.017829,0.651018,-0.825886,-0.016756,-0.433416,-0.068172,0.117703,0.132541,0.711179,...,-0.625861,-1.038187,-1.117591,-0.173885,-0.052019,-1.652902,1.451063,0.064013,-0.226102,-0.619278
3,0.059696,0.107079,-0.935282,-0.562332,-1.138087,-0.547036,-0.201778,-0.401271,2.655044,0.017943,...,-0.288925,0.347900,0.628829,-0.173885,-0.052019,0.452675,-3.794050,-0.767526,-0.226102,-0.811634
4,-0.467137,-0.473268,1.776515,1.007943,1.306532,-0.433416,1.122939,2.038003,0.035973,-0.001205,...,-0.453735,-0.411870,-0.031423,-0.173885,-0.052019,-0.400859,0.820233,0.103539,-0.226102,-0.603137
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17265,-0.287299,-0.265062,0.651018,-0.309613,-0.466344,-0.376607,-0.328954,-0.762645,0.205906,-0.318404,...,0.601891,-0.863756,-0.099034,-0.173885,-0.052019,-0.181793,-0.186722,-0.664175,-0.226102,-1.105211
17266,1.544852,1.483783,0.651018,-0.302714,-0.020570,1.725359,-1.126782,-1.525227,0.287451,1.990773,...,1.378486,-0.236316,-1.293851,-0.173885,-0.052019,-1.272525,-0.312335,1.922099,15.437718,0.515242
17267,-0.184606,-0.195669,-0.935282,-0.853742,0.573928,-0.376607,-0.462918,0.518590,-0.502200,-0.058064,...,0.774520,-1.445029,1.910897,-0.173885,-0.052019,-0.394160,0.604883,-0.252131,-0.226102,0.337319
17268,-0.956427,-0.973556,-0.935282,-0.052975,-0.546289,-0.717466,-0.120765,1.044225,-1.711059,0.954084,...,-0.401251,0.061317,-0.047990,-0.173885,-0.052019,1.230196,-0.481137,-0.809969,-0.226102,0.450469


In [ ]:
X_valid = alll[17270:17270+4317].reset_index().iloc[:,1:];X_valid

,0,1,2,3,4,5,6,7,8,9,...,1342,1343,1344,1345,1346,1347,1348,1349,1350,1351
0,-0.787809,-0.784793,1.776515,-1.515765,-0.253649,-0.603846,-0.618401,-1.364936,-0.589613,-0.443720,...,0.820122,0.041372,0.084913,-0.173885,-0.052019,1.375272,-0.748232,-0.480476,-0.226102,-0.412407
1,-0.732479,-0.691400,1.776515,-0.230559,0.316981,-0.717466,-0.648509,-0.204158,-0.453358,-1.519076,...,0.073496,-0.780648,-0.727361,-0.173885,-0.052019,-0.532140,-0.624092,0.678645,-0.226102,-0.151467
2,0.609539,0.604508,1.776515,1.120299,-0.637803,0.873211,2.151537,-0.180315,0.671454,-0.834309,...,0.711795,-0.748443,0.941118,-0.173885,-0.052019,-0.254329,0.078914,2.182550,0.314029,0.906395
3,0.083581,0.092450,-0.935282,-0.231692,1.158360,-0.660656,-0.385895,-1.124019,0.573312,0.004467,...,-0.479826,0.581924,-0.351025,-0.173885,-0.052019,0.578840,-1.000637,-0.754664,-0.226102,-0.065195
4,-0.895284,-0.893081,-0.935282,-0.343816,1.396820,-0.603846,-0.790658,-0.401271,-0.882512,0.885945,...,-0.018084,0.273483,0.388473,-0.173885,-0.052019,0.140315,0.197199,-0.686433,-0.226102,-1.784368
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4312,-0.273649,-0.270357,0.651018,-0.680033,0.248866,1.270880,-0.563737,0.549713,-0.384369,-0.905223,...,0.291786,0.451535,-0.600156,-0.173885,-0.052019,0.705455,-0.372433,1.434664,-0.226102,-0.433346
4313,-1.160881,-1.154148,-0.935282,-0.718430,-2.626363,-0.319797,-0.624261,2.413642,-1.154947,-0.671175,...,0.290958,-0.763500,0.140508,-0.173885,-0.052019,1.447736,-2.491760,-0.316799,-0.226102,-0.479245
4314,0.611911,0.591031,0.651018,-0.281634,-0.279290,0.191492,-0.736377,-0.401271,0.009516,-1.161780,...,1.276143,-0.318450,0.042843,-0.173885,-0.052019,-0.844246,-0.260666,-0.291010,0.314029,0.906989
4315,-0.781984,-0.772776,-0.935282,-1.195966,-0.413790,-0.092557,-0.276245,-0.067695,-0.222001,0.592184,...,0.043353,-0.907532,-0.013474,-0.173885,-0.052019,0.533160,0.678769,-0.508825,-0.226102,-0.372489


In [ ]:
X_test = alll[17270+4317:].reset_index().iloc[:,1:];X_test

,0,1,2,3,4,5,6,7,8,9,...,1342,1343,1344,1345,1346,1347,1348,1349,1350,1351
0,-0.376033,-0.363430,0.651018,-0.307479,0.592681,-0.603846,-0.596291,-0.282468,-0.111349,-1.363724,...,0.182574,0.199803,0.203688,-0.173885,-0.052019,-0.810005,-0.914305,0.043472,-0.226102,0.386602
1,-0.347738,-0.346473,0.651018,-0.613063,0.236143,1.498119,0.397149,0.084169,0.000727,-0.615028,...,-0.579818,0.205294,-0.087941,-0.173885,-0.052019,0.637444,0.127584,2.072007,-0.226102,-0.396176
2,0.235198,0.236824,-0.935282,-0.700442,-0.543580,0.248302,-0.359879,-0.429205,-0.538349,0.090459,...,1.159775,-0.247249,1.511214,-0.173885,-0.052019,-0.026194,-1.112096,0.616602,0.314029,0.783705
3,-0.396507,-0.396673,-0.935282,0.398871,-1.973844,-0.774276,0.297125,1.044225,-2.604744,-1.249817,...,-1.761246,-0.113249,-0.040132,-0.173885,-0.052019,1.157019,2.694395,-0.843760,-0.226102,0.088795
4,-0.365554,-0.348710,0.651018,-0.454141,0.852007,-0.660656,-0.103120,0.080561,-1.554970,-0.701272,...,0.884269,1.075930,0.935844,-0.173885,-0.052019,1.720518,0.532167,-0.669813,-0.226102,0.923578
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14375,2.434637,2.470233,-0.935282,2.444313,0.085558,-0.774276,-0.421931,1.044225,1.073118,-0.932893,...,-1.149085,0.184284,0.821311,-0.173885,-0.052019,-0.328714,0.572698,-0.772905,-0.226102,1.655081
14376,-0.357541,-0.357676,-0.935282,0.398871,3.101870,-0.717466,0.297125,1.044225,-2.228148,-0.818197,...,-2.168055,2.764155,0.481166,-0.173885,-0.052019,2.188832,-0.447706,-0.851536,-0.226102,0.737441
14377,-0.165235,-0.198549,-0.935282,-0.075443,1.098311,-0.774276,-0.238699,1.044225,-0.253429,-1.827729,...,-1.182235,2.204344,-0.363499,-0.173885,-0.052019,0.663401,0.070965,-0.809742,-0.226102,-0.254768
14378,3.405554,3.431798,-0.935282,2.000310,-0.181798,-0.717466,2.146455,-1.124019,2.033663,1.429454,...,-0.081216,-1.098615,0.837121,-0.173885,-0.052019,-0.854609,0.400064,-0.744621,-0.226102,-0.337506


### Step 2: Define the hyper-model

In [ ]:
def model_fn(hp):
    inputs = keras.Input(shape=(X_train.shape[1],))
    x = inputs
    for i in range(hp.Int('num_layers', 2, 3)):
        x = keras.layers.Dense(hp.Int('unit_'+str(i), 16, 64, step=16), activation='relu')(x)
        x = keras.layers.Dropout(hp.Float('dropout_'+str(i), 0, 0.5, step=0.25, default=0.5))(x)
    outputs = keras.layers.Dense(8, activation='softmax')(x) # 예측값이 8종류이므로 8개 출력 뉴런 필요
    model = keras.Model(inputs, outputs)
    model.compile(loss='categorical_crossentropy', # Multiclass Classification에서 사용하는 loss function
                  optimizer=tf.keras.optimizers.Adam(hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])), 
                  metrics=[keras.metrics.CategoricalCrossentropy()]) # Multiclass Classification에서 사용하는 평가지표
    return model

### Step 3: Build multiple hyper-tuned models

In [ ]:
# 
def dnn():
  tuner = kt.Hyperband(model_fn,
                      objective=kt.Objective('val_categorical_crossentropy', direction="min"), 
                      max_epochs=20,
                      hyperband_iterations=2,
                      overwrite=True,
                      directory='dnn_tuning')
  tuner.search(X_train, y_train, validation_data=(X_valid, y_valid), 
              callbacks=[tf.keras.callbacks.EarlyStopping(patience=1)])
  model = tuner.get_best_models(1)[0]  
  tuner.results_summary(1)
  t = pd.Timestamp.now()
  fname = f"dnn_submission_{t.month:02}{t.day:02}1.csv"
  pred = pd.DataFrame(model.predict(X_test))
  pred.columns = ['F20','F30','F40','F50','M20','M30','M40','M50']

  pd.concat([pd.Series(IDtest.custid, name="ID"), pred] ,axis=1).to_csv('/content/drive/MyDrive/머러컴피티션/진짜마지막/dnn결과/'+fname, index=False)
  print(f"'{fname}' is ready to submit.")
for i in range(10):
  dnn()

Trial 57 Complete [00h 00m 43s]
val_categorical_crossentropy: 1.6017296314239502

Best val_categorical_crossentropy So Far: 1.4960705041885376
Total elapsed time: 00h 09m 44s

Search: Running Trial #58

Value             |Best Value So Far |Hyperparameter
3                 |3                 |num_layers
64                |64                |unit_0
0.5               |0.25              |dropout_0
16                |48                |unit_1
0.25              |0                 |dropout_1
0.0001            |0.001             |learning_rate
48                |32                |unit_2
0.25              |0                 |dropout_2
20                |20                |tuner/epochs
0                 |7                 |tuner/initial_epoch
0                 |1                 |tuner/bracket
0                 |1                 |tuner/round

Epoch 1/20
540/540 [==============================] - 3s 4ms/step - loss: 2.0796 - categorical_crossentropy: 2.0796 - val_loss: 1.7875 - val_categorical

KeyboardInterrupt: ignored

In [ ]:
t = pd.Timestamp.now()
fname = f"dnn_submission_{t.month:02}{t.day:02}.1.490.csv"
pred = pd.DataFrame(model.predict(X_test))
pred.columns = ['F20','F30','F40','F50','M20','M30','M40','M50']

pd.concat([pd.Series(IDtest.custid, name="ID"), pred] ,axis=1).to_csv('/content/drive/MyDrive/머러컴피티션/진짜마지막/dnn결과/'+fname, index=False)
print(f"'{fname}' is ready to submit.")

In [ ]:
# 
tuner = kt.Hyperband(model_fn,
                     objective=kt.Objective('val_categorical_crossentropy', direction="min"), 
                     max_epochs=20,
                     hyperband_iterations=2,
                     overwrite=True,
                     directory='dnn_tuning')
tuner.search(X_train, y_train, validation_data=(X_valid, y_valid), 
             callbacks=[tf.keras.callbacks.EarlyStopping(patience=1)])
model = tuner.get_best_models(1)[0]  
tuner.results_summary(1)

Trial 60 Complete [00h 00m 25s]
val_categorical_crossentropy: 1.5098053216934204

Best val_categorical_crossentropy So Far: 1.4994770288467407
Total elapsed time: 00h 12m 26s
INFO:tensorflow:Oracle triggered exit
Results summary
Results in dnn_tuning/untitled_project
Showing 1 best trials
Trial summary
Hyperparameters:
num_layers: 3
unit_0: 32
dropout_0: 0.25
unit_1: 80
dropout_1: 0.0
unit_2: 32
dropout_2: 0.25
learning_rate: 0.001
unit_3: 48
dropout_3: 0.25
tuner/epochs: 20
tuner/initial_epoch: 0
tuner/bracket: 0
tuner/round: 0
Score: 1.4994770288467407


In [ ]:
t = pd.Timestamp.now()
fname = f"dnn_submission_{t.month:02}{t.day:02}.1.490.csv"
pred = pd.DataFrame(model.predict(X_test))
pred.columns = ['F20','F30','F40','F50','M20','M30','M40','M50']

pd.concat([pd.Series(IDtest.custid, name="ID"), pred] ,axis=1).to_csv('/content/drive/MyDrive/머러컴피티션/submission/snd_sum_shap_0612.csv/'+fname, index=False)
print(f"'{fname}' is ready to submit.")

In [ ]:
dnn487.to_csv('/content/drive/MyDrive/머러컴피티션/submission/snd_sum_shap_0612.csv/'+'dnn487', index=False)

In [ ]:
# second tuning 전 step=0.25,max_epochs=20, num 4 3 16 48 6
tuner = kt.Hyperband(model_fn,
                     objective=kt.Objective('val_categorical_crossentropy', direction="min"), 
                     max_epochs=20,
                     hyperband_iterations=2,
                     overwrite=True,
                     directory='dnn_tuning')
tuner.search(X_train, y_train, validation_data=(X_valid, y_valid), 
             callbacks=[tf.keras.callbacks.EarlyStopping(patience=1)])
model = tuner.get_best_models(1)[0]  
tuner.results_summary(1)

Trial 60 Complete [00h 00m 52s]
val_categorical_crossentropy: 1.530590295791626

Best val_categorical_crossentropy So Far: 1.4903638362884521
Total elapsed time: 00h 11m 44s
INFO:tensorflow:Oracle triggered exit
Results summary
Results in dnn_tuning/untitled_project
Showing 1 best trials
Trial summary
Hyperparameters:
num_layers: 3
unit_0: 48
dropout_0: 0.25
unit_1: 64
dropout_1: 0.25
unit_2: 48
dropout_2: 0.0
learning_rate: 0.001
tuner/epochs: 20
tuner/initial_epoch: 7
tuner/bracket: 1
tuner/round: 1
tuner/trial_id: 0022
Score: 1.4903638362884521


In [ ]:
t = pd.Timestamp.now()
fname = f"dnn_submission_{t.month:02}{t.day:02}.1.490.csv"
pred = pd.DataFrame(model.predict(X_test))
pred.columns = ['F20','F30','F40','F50','M20','M30','M40','M50']

In [ ]:

pd.concat([pd.Series(IDtest.custid, name="ID"), pred] ,axis=1).to_csv('/content/drive/MyDrive/머러컴피티션/submission/snd_sum_shap_0612.csv/'+fname, index=False)
print(f"'{fname}' is ready to submit.")

'dnn_submission_0615.1.490.csv' is ready to submit.


In [ ]:
# second tuning 전 step=0.25,max_epochs=20, num 3 80
tuner = kt.Hyperband(model_fn,
                     objective=kt.Objective('val_categorical_crossentropy', direction="min"), 
                     max_epochs=20,
                     hyperband_iterations=2,
                     overwrite=True,
                     directory='dnn_tuning')
tuner.search(X_train, y_train, validation_data=(X_valid, y_valid), 
             callbacks=[tf.keras.callbacks.EarlyStopping(patience=1)])
model = tuner.get_best_models(1)[0]  
tuner.results_summary(1)

Trial 60 Complete [00h 00m 20s]
val_categorical_crossentropy: 1.502875566482544

Best val_categorical_crossentropy So Far: 1.48936927318573
Total elapsed time: 00h 11m 10s
INFO:tensorflow:Oracle triggered exit
Results summary
Results in dnn_tuning/untitled_project
Showing 1 best trials
Trial summary
Hyperparameters:
num_layers: 3
unit_0: 48
dropout_0: 0.25
unit_1: 64
dropout_1: 0.0
unit_2: 48
dropout_2: 0.25
learning_rate: 0.001
tuner/epochs: 20
tuner/initial_epoch: 7
tuner/bracket: 2
tuner/round: 2
tuner/trial_id: 0042
Score: 1.48936927318573


In [ ]:
t = pd.Timestamp.now()
fname = f"dnn_submission_{t.month:02}{t.day:02}.1.489.csv"
pred = pd.DataFrame(model.predict(X_test))
pred.columns = ['F20','F30','F40','F50','M20','M30','M40','M50']
pd.concat([pd.Series(IDtest.custid, name="ID"), pred] ,axis=1).to_csv('/content/drive/MyDrive/머러컴피티션/submission/snd_sum_shap_0612.csv/'+fname, index=False)
print(f"'{fname}' is ready to submit.")

'dnn_submission_0615061.489.csv' is ready to submit.


In [ ]:
# second tuning 전 step=0.25,max_epochs=20,
tuner = kt.Hyperband(model_fn,
                     objective=kt.Objective('val_categorical_crossentropy', direction="min"), 
                     max_epochs=20,
                     hyperband_iterations=2,
                     overwrite=True,
                     directory='dnn_tuning')
tuner.search(X_train, y_train, validation_data=(X_valid, y_valid), 
             callbacks=[tf.keras.callbacks.EarlyStopping(patience=1)])
model = tuner.get_best_models(1)[0]  
tuner.results_summary(1)

Trial 60 Complete [00h 00m 09s]
val_categorical_crossentropy: 1.566907525062561

Best val_categorical_crossentropy So Far: 1.4984831809997559
Total elapsed time: 00h 10m 24s
INFO:tensorflow:Oracle triggered exit
Results summary
Results in dnn_tuning/untitled_project
Showing 1 best trials
Trial summary
Hyperparameters:
num_layers: 2
unit_0: 32
dropout_0: 0.25
unit_1: 32
dropout_1: 0.25
learning_rate: 0.001
unit_2: 48
dropout_2: 0.5
tuner/epochs: 20
tuner/initial_epoch: 7
tuner/bracket: 2
tuner/round: 2
tuner/trial_id: 0043
Score: 1.4984831809997559


In [ ]:
# second tuning 전 step=0.2,max_epochs=20,
tuner = kt.Hyperband(model_fn,
                     objective=kt.Objective('val_categorical_crossentropy', direction="min"), 
                     max_epochs=20,
                     hyperband_iterations=2,
                     overwrite=True,
                     directory='dnn_tuning')
tuner.search(X_train, y_train, validation_data=(X_valid, y_valid), 
             callbacks=[tf.keras.callbacks.EarlyStopping(patience=1)])
model = tuner.get_best_models(1)[0]  
tuner.results_summary(1)

Trial 60 Complete [00h 00m 06s]
val_categorical_crossentropy: 1.695425271987915

Best val_categorical_crossentropy So Far: 1.493905782699585
Total elapsed time: 00h 10m 19s
INFO:tensorflow:Oracle triggered exit
Results summary
Results in dnn_tuning/untitled_project
Showing 1 best trials
Trial summary
Hyperparameters:
num_layers: 2
unit_0: 48
dropout_0: 0.4
unit_1: 16
dropout_1: 0.2
learning_rate: 0.001
unit_2: 32
dropout_2: 0.2
tuner/epochs: 7
tuner/initial_epoch: 0
tuner/bracket: 1
tuner/round: 0
Score: 1.493905782699585


In [ ]:
# second tuning 전 step=0.25,max_epochs=20,
tuner = kt.Hyperband(model_fn,
                     objective=kt.Objective('val_categorical_crossentropy', direction="min"), 
                     max_epochs=20,
                     hyperband_iterations=2,
                     overwrite=True,
                     directory='dnn_tuning')
tuner.search(X_train, y_train, validation_data=(X_valid, y_valid), 
             callbacks=[tf.keras.callbacks.EarlyStopping(patience=1)])
model = tuner.get_best_models(1)[0]  
tuner.results_summary(1)

Trial 60 Complete [00h 00m 05s]
val_categorical_crossentropy: 1.5981625318527222

Best val_categorical_crossentropy So Far: 1.4822351932525635
Total elapsed time: 00h 09m 06s
INFO:tensorflow:Oracle triggered exit
Results summary
Results in dnn_tuning/untitled_project
Showing 1 best trials
Trial summary
Hyperparameters:
num_layers: 2
unit_0: 64
dropout_0: 0.5
unit_1: 48
dropout_1: 0.0
learning_rate: 0.001
unit_2: 16
dropout_2: 0.25
tuner/epochs: 20
tuner/initial_epoch: 7
tuner/bracket: 2
tuner/round: 2
tuner/trial_id: 0042
Score: 1.4822351932525635


In [ ]:
# second tuning 후 step=0.25,max_epochs=20,
tuner = kt.Hyperband(model_fn,
                     objective=kt.Objective('val_categorical_crossentropy', direction="min"), 
                     max_epochs=20,
                     hyperband_iterations=2,
                     overwrite=True,
                     directory='dnn_tuning')
tuner.search(X_train, y_train, validation_data=(X_valid, y_valid), 
             callbacks=[tf.keras.callbacks.EarlyStopping(patience=1)])
model = tuner.get_best_models(1)[0]  
tuner.results_summary(1)

Trial 60 Complete [00h 00m 18s]
val_categorical_crossentropy: 1.5091098546981812

Best val_categorical_crossentropy So Far: 1.491424560546875
Total elapsed time: 00h 08m 21s
INFO:tensorflow:Oracle triggered exit
Results summary
Results in dnn_tuning/untitled_project
Showing 1 best trials
Trial summary
Hyperparameters:
num_layers: 2
unit_0: 16
dropout_0: 0.2
unit_1: 16
dropout_1: 0.2
learning_rate: 0.001
unit_2: 16
dropout_2: 0.4
tuner/epochs: 20
tuner/initial_epoch: 7
tuner/bracket: 1
tuner/round: 1
tuner/trial_id: 0023
Score: 1.491424560546875


In [ ]:
# second tuning 후 256
tuner = kt.Hyperband(model_fn,
                     objective=kt.Objective('val_categorical_crossentropy', direction="min"), 
                     max_epochs=10,
                     hyperband_iterations=2,
                     overwrite=True,
                     directory='dnn_tuning')
tuner.search(X_train, y_train, validation_data=(X_valid, y_valid), 
             callbacks=[tf.keras.callbacks.EarlyStopping(patience=1)])
model = tuner.get_best_models(1)[0]  
tuner.results_summary(1)

Trial 60 Complete [00h 00m 15s]
val_categorical_crossentropy: 1.4941835403442383

Best val_categorical_crossentropy So Far: 1.4926292896270752
Total elapsed time: 00h 07m 42s
INFO:tensorflow:Oracle triggered exit
Results summary
Results in dnn_tuning/untitled_project
Showing 1 best trials
Trial summary
Hyperparameters:
num_layers: 5
unit_0: 224
dropout_0: 0.2
unit_1: 208
dropout_1: 0.2
learning_rate: 0.0001
unit_2: 64
dropout_2: 0.1
unit_3: 208
dropout_3: 0.0
unit_4: 176
dropout_4: 0.1
unit_5: 96
dropout_5: 0.4
tuner/epochs: 10
tuner/initial_epoch: 4
tuner/bracket: 2
tuner/round: 2
tuner/trial_id: 0012
Score: 1.4926292896270752


In [ ]:
# 1st tuning
tuner = kt.Hyperband(model_fn,
                     objective=kt.Objective('val_categorical_crossentropy', direction="min"), 
                     max_epochs=20,
                     hyperband_iterations=2,
                     overwrite=True,
                     directory='dnn_tuning')
tuner.search(X_train, y_train, validation_data=(X_valid, y_valid), 
             callbacks=[tf.keras.callbacks.EarlyStopping(patience=1)])
model = tuner.get_best_models(1)[0]  
tuner.results_summary(1)

Trial 60 Complete [00h 00m 04s]
val_categorical_crossentropy: 1.7119332551956177

Best val_categorical_crossentropy So Far: 1.4861210584640503
Total elapsed time: 00h 08m 21s
INFO:tensorflow:Oracle triggered exit
Results summary
Results in dnn_tuning/untitled_project
Showing 1 best trials
Trial summary
Hyperparameters:
num_layers: 5
unit_0: 224
dropout_0: 0.2
unit_1: 208
dropout_1: 0.2
learning_rate: 0.0001
unit_2: 64
dropout_2: 0.1
unit_3: 208
dropout_3: 0.0
unit_4: 176
dropout_4: 0.1
tuner/epochs: 20
tuner/initial_epoch: 7
tuner/bracket: 2
tuner/round: 2
tuner/trial_id: 0043
Score: 1.4861210584640503


In [ ]:
X_train

array([[-0.4313704 , -0.43976308,  0.66140269, ..., -0.49763251,
        -0.22898994,  0.04961078],
       [-0.3807711 , -0.38066675,  0.66140269, ..., -0.54307493,
        -0.22898994,  0.04199081],
       [-0.03608654, -0.02028264,  0.66140269, ...,  0.06156525,
        -0.22898994, -0.63616637],
       ...,
       [-0.19590416, -0.20739021, -0.92168023, ..., -0.25070737,
        -0.22898994,  0.33706808],
       [-1.00836984, -1.02581696, -0.92168023, ..., -0.80171269,
        -0.22898994,  0.45218674],
       [-0.8884726 , -0.88631277,  0.66140269, ...,  3.99275047,
         2.51577423, -0.10468711]])

In [ ]:
t = pd.Timestamp.now()
fname = f"dnn_submission_{t.month:02}{t.day:02}{t.hour:02}1.493.csv"
pred = pd.DataFrame(model.predict(X_test))
pred.columns = ['F20','F30','F40','F50','M20','M30','M40','M50']
pd.concat([pd.Series(IDtest.custid, name="ID"), pred] ,axis=1).to_csv('/content/drive/MyDrive/머러컴피티션/submission/snd_sum_shap_0612.csv/'+fname, index=False)
print(f"'{fname}' is ready to submit.")

'dnn_submission_0615051.493.csv' is ready to submit.


In [ ]:
pd.read_csv('/content/drive/MyDrive/머러컴피티션/submission/dnn+lgbm/dnn_submission_06131301.csv')

,ID,F20,F30,F40,F50,M20,M30,M40,M50
0,30001,0.183698,0.138087,0.037649,0.116351,0.123118,0.209439,0.050899,0.140758
1,30002,0.005879,0.047781,0.325411,0.010709,0.001529,0.017993,0.572350,0.018347
2,30003,0.717163,0.186987,0.018404,0.017040,0.022689,0.023989,0.005746,0.007980
3,30005,0.513325,0.112715,0.106024,0.037476,0.065985,0.126673,0.016714,0.021089
4,30007,0.917493,0.060677,0.002505,0.002173,0.007393,0.007921,0.000681,0.001157
...,...,...,...,...,...,...,...,...,...
14375,49988,0.070449,0.443913,0.020704,0.007512,0.004949,0.367313,0.080524,0.004636
14376,49990,0.448465,0.257474,0.149285,0.020725,0.032518,0.051643,0.009137,0.030754
14377,49992,0.392092,0.126545,0.054908,0.073504,0.098653,0.178131,0.036839,0.039328
14378,49993,0.024272,0.649988,0.140282,0.004214,0.002694,0.129725,0.046946,0.001879


In [ ]:
pd.read_csv('/content/drive/MyDrive/머러컴피티션/submission/dnn+lgbm/dnn_submission_0613_1.512.csv')

,ID,F20,F30,F40,F50,M20,M30,M40,M50
0,30001,0.183698,0.138087,0.037649,0.116351,0.123118,0.209439,0.050899,0.140758
1,30002,0.005879,0.047781,0.325411,0.010709,0.001529,0.017993,0.572350,0.018347
2,30003,0.717163,0.186987,0.018404,0.017040,0.022689,0.023989,0.005746,0.007980
3,30005,0.513325,0.112715,0.106024,0.037476,0.065985,0.126673,0.016714,0.021089
4,30007,0.917493,0.060677,0.002505,0.002173,0.007393,0.007921,0.000681,0.001157
...,...,...,...,...,...,...,...,...,...
14375,49988,0.070449,0.443913,0.020704,0.007512,0.004949,0.367313,0.080524,0.004636
14376,49990,0.448465,0.257474,0.149285,0.020725,0.032518,0.051643,0.009137,0.030754
14377,49992,0.392092,0.126545,0.054908,0.073504,0.098653,0.178131,0.036839,0.039328
14378,49993,0.024272,0.649988,0.140282,0.004214,0.002694,0.129725,0.046946,0.001879


In [ ]:
pd.concat([pd.Series(IDtest.custid, name="ID"), pred] ,axis=1)

,ID,F20,F30,F40,F50,M20,M30,M40,M50
0,30001,0.127368,0.127141,0.070802,0.215573,0.076733,0.153606,0.043525,0.185252
1,30002,0.003001,0.047245,0.472640,0.029603,0.000180,0.016012,0.351101,0.080218
2,30003,0.782996,0.143690,0.021967,0.012854,0.013883,0.015447,0.002693,0.006471
3,30005,0.753075,0.105907,0.028009,0.022648,0.051321,0.023162,0.004991,0.010888
4,30007,0.886505,0.057098,0.005901,0.007949,0.031386,0.007237,0.000590,0.003335
...,...,...,...,...,...,...,...,...,...
14375,49988,0.178404,0.602676,0.026653,0.005637,0.007521,0.163109,0.011531,0.004470
14376,49990,0.455819,0.259807,0.094585,0.043054,0.029502,0.077526,0.014949,0.024758
14377,49992,0.328283,0.147437,0.089196,0.109694,0.114513,0.106707,0.031770,0.072399
14378,49993,0.046496,0.657718,0.036445,0.004434,0.002994,0.204592,0.042543,0.004780


In [ ]:
pd.read_csv('dnn_submission_06131301.csv')

FileNotFoundError: ignored

In [ ]:
pip install pytorch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Operation cancelled by user


In [ ]:
import torch

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
class MLP(torch.nn.Module):
  def __init__(self):
    super().__init__()
    self.linear1 = torch.nn.Linear(1352, 160)
    self.linear2 = torch.nn.Linear(160, 256)
    self.linear3 = torch.nn.Linear(256, 160)
    self.linear4 = torch.nn.Linear(160, 112)
    self.linear5 = torch.nn.Linear(112,8)

    self.drop_p1 = torch.nn.Dropout(.4)
    self.drop_p2 = torch.nn.Dropout(.5)
    self.drop_p3 = torch.nn.Dropout(.1)
    self.drop_p4 = torch.nn.Dropout(.1)
  def forward(self, x):
    x = self.linear1(x)
    x = self.drop_p1(x)
    x = self.linear2(x)
    x = self.drop_p2(x)
    x = self.linear3(x)
    x = self.drop_p3(x)
    x = self.linear4(x)
    x = self.drop_p4(x)
    x = self.linear5(x)
    x = F.softmax(x)
    return x

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
model = MLP().to(device)
optimizer = torch.optim.Adam(model.parameters(), 0.001)
loss = torch.nn.CrossEntropyLoss()
loss_val = torch.nn.CrossEntropyLoss()

In [ ]:
X_train = X_train.to_numpy()
X_valid = X_valid.to_numpy()

AttributeError: ignored

In [ ]:
X_test = X_test.to_numpy()

In [ ]:
X_train, y_train, X_valid, y_valid, X_test = torch.FloatTensor(X_train), torch.tensor(y_train), torch.FloatTensor(X_valid), torch.tensor(y_valid), torch.FloatTensor(X_test)
X_train, y_train, X_valid, y_valid, X_test = X_train.to(device), y_train.argmax(axis=1).to(device), X_valid.to(device), y_valid.argmax(axis=1).to(device), X_test.to(device)

In [ ]:
for epoch in range(100):
  model.train()
  optimizer.zero_grad()
  output = model(X_train)
  loss_output = loss(output, y_train)
  loss_output.backward()
  optimizer.step()
  with torch.no_grad():
    model.eval()
    o = model(X_valid)
    loss_val_o = loss_val(o, y_valid)

    print(f'Epoch {epoch}: Train Loss - {loss_output.item()}, Valid Loss - {loss_val_o.item()}')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Epoch 0: Train Loss - 2.0748913288116455, Valid Loss - 2.054661273956299
Epoch 1: Train Loss - 2.0538861751556396, Valid Loss - 2.0130107402801514
Epoch 2: Train Loss - 2.01206111907959, Valid Loss - 1.9559757709503174
Epoch 3: Train Loss - 1.9554836750030518, Valid Loss - 1.9128079414367676
Epoch 4: Train Loss - 1.9160032272338867, Valid Loss - 1.8861110210418701
Epoch 5: Train Loss - 1.8915938138961792, Valid Loss - 1.8756519556045532
Epoch 6: Train Loss - 1.881231427192688, Valid Loss - 1.868463397026062
Epoch 7: Train Loss - 1.8727240562438965, Valid Loss - 1.8633955717086792
Epoch 8: Train Loss - 1.869883418083191, Valid Loss - 1.859413504600525
Epoch 9: Train Loss - 1.8639237880706787, Valid Loss - 1.8559777736663818
Epoch 10: Train Loss - 1.8608778715133667, Valid Loss - 1.8546075820922852
Epoch 11: Train Loss - 1.8588643074035645, Valid Loss - 1.8520416021347046
Epoch 12: Train Loss - 1.8567801713943481, Valid Loss - 1.850265622138977
Epoch 13: Train Loss - 1.853916049003601, V

In [ ]:
X_test.numpy()

TypeError: ignored

In [ ]:
IDtest['ID'] = IDtest.custid;IDtest

,custid,ID
0,30001,30001
1,30002,30002
2,30003,30003
3,30005,30005
4,30007,30007
...,...,...
14375,49988,49988
14376,49990,49990
14377,49992,49992
14378,49993,49993


In [ ]:
model(X_test)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


tensor([[9.9846e-08, 3.0329e-13, 8.3819e-10,  ..., 8.8881e-01, 6.2200e-06,
         1.0853e-01],
        [2.1975e-23, 2.1053e-08, 1.0000e+00,  ..., 1.4869e-13, 6.9779e-11,
         5.9568e-10],
        [1.0000e+00, 3.6850e-23, 1.8818e-23,  ..., 2.1019e-27, 3.1245e-21,
         7.3889e-28],
        ...,
        [1.0000e+00, 3.9195e-22, 1.5885e-24,  ..., 2.5822e-16, 1.6440e-16,
         6.1158e-18],
        [0.0000e+00, 1.0000e+00, 9.3743e-26,  ..., 3.0501e-22, 4.3863e-29,
         6.0033e-32],
        [1.0000e+00, 4.3513e-41, 8.8627e-35,  ..., 8.0623e-34, 3.9529e-29,
         9.7782e-35]], device='cuda:0', grad_fn=<SoftmaxBackward0>)

### Step 4: Make submissions

In [ ]:
pd.DataFrame(model(X_test).cpu().detach().numpy())

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


,0,1,2,3,4,5,6,7
0,9.984588e-08,3.032883e-13,8.381869e-10,1.199083e-03,1.454526e-03,8.888149e-01,6.219997e-06,1.085251e-01
1,2.197522e-23,2.105274e-08,1.000000e+00,7.951067e-13,1.204862e-11,1.486880e-13,6.977947e-11,5.956844e-10
2,1.000000e+00,3.684961e-23,1.881814e-23,9.322462e-28,4.805037e-21,2.101915e-27,3.124468e-21,7.388923e-28
3,1.000000e+00,1.798595e-30,7.093066e-38,5.802855e-38,3.100480e-31,1.288373e-37,1.435475e-30,8.577096e-40
4,1.000000e+00,8.758115e-43,1.808516e-41,2.993454e-41,9.518312e-34,2.606415e-43,3.994961e-35,9.668959e-44
...,...,...,...,...,...,...,...,...
14375,5.061462e-36,1.000000e+00,2.208607e-38,2.012189e-36,4.924792e-34,1.729695e-19,7.487833e-29,3.039730e-34
14376,1.000000e+00,9.736440e-29,4.814074e-17,4.344541e-18,3.874806e-16,4.048153e-25,7.032839e-18,2.574937e-20
14377,1.000000e+00,3.919467e-22,1.588543e-24,4.033515e-15,2.529584e-15,2.582201e-16,1.644016e-16,6.115763e-18
14378,0.000000e+00,1.000000e+00,9.374287e-26,1.749261e-36,2.674072e-34,3.050133e-22,4.386348e-29,6.003314e-32


In [ ]:
t = pd.Timestamp.now()
fname = f"MLP_submission_{t.month:02}{t.day:02}{t.hour:02}{t.minute:02}.csv"
pred = pd.DataFrame(model(X_test).cpu().detach().numpy())
pred.columns = ['F20','F30','F40','F50','M20','M30','M40','M50']
pd.concat([IDtest.ID, pred] ,axis=1).to_csv(fname, index=False)
print(f"'{fname}' is ready to submit.")

'MLP_submission_06131326.csv' is ready to submit.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


In [ ]:
pd.concat([IDtest.ID, pred] ,axis=1)

,ID,F20,F30,F40,F50,M20,M30,M40,M50
0,30001,9.984588e-08,3.032883e-13,8.381869e-10,1.199083e-03,1.454526e-03,8.888149e-01,6.219997e-06,1.085251e-01
1,30002,2.197522e-23,2.105274e-08,1.000000e+00,7.951067e-13,1.204862e-11,1.486880e-13,6.977947e-11,5.956844e-10
2,30003,1.000000e+00,3.684961e-23,1.881814e-23,9.322462e-28,4.805037e-21,2.101915e-27,3.124468e-21,7.388923e-28
3,30005,1.000000e+00,1.798595e-30,7.093066e-38,5.802855e-38,3.100480e-31,1.288373e-37,1.435475e-30,8.577096e-40
4,30007,1.000000e+00,8.758115e-43,1.808516e-41,2.993454e-41,9.518312e-34,2.606415e-43,3.994961e-35,9.668959e-44
...,...,...,...,...,...,...,...,...,...
14375,49988,5.061462e-36,1.000000e+00,2.208607e-38,2.012189e-36,4.924792e-34,1.729695e-19,7.487833e-29,3.039730e-34
14376,49990,1.000000e+00,9.736440e-29,4.814074e-17,4.344541e-18,3.874806e-16,4.048153e-25,7.032839e-18,2.574937e-20
14377,49992,1.000000e+00,3.919467e-22,1.588543e-24,4.033515e-15,2.529584e-15,2.582201e-16,1.644016e-16,6.115763e-18
14378,49993,0.000000e+00,1.000000e+00,9.374287e-26,1.749261e-36,2.674072e-34,3.050133e-22,4.386348e-29,6.003314e-32


In [ ]:
pd.concat([IDtest.ID, pred] ,axis=1)

t = pd.Timestamp.now()
fname = f"dnn_submission_{t.month:02}{t.day:02}{t.hour:02}{t.minute:02}.csv"
pred = pd.DataFrame(model.predict(X_test))
pred.columns = ['F20','F30','F40','F50','M20','M30','M40','M50']
pd.concat([pd.Series(IDtest, name="ID"), pred] ,axis=1).to_csv(fname, index=False)
print(f"'{fname}' is ready to submit.")

<font color="#CC3D3D"><p>
# End